In [ ]:
#!pip install simple_salesforce

In [1]:
import json
from sqlalchemy import create_engine
from datetime import date
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
# Make sure to use your own `config.py` file. Consider ensuring that these variable names are in sync
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [3]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [4]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Prepare ETL for the Course data

In [5]:
staff_data_df = pd.read_sql("SELECT * FROM staff", conn)
staff_data_df

,ID_Staff,EmployeeID,LastName,FirstName,MiddleName,BirthDate
0,1,000184220,Wimberly,Sam,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Popelka,Sarah,Nisan,1994-06-07


In [6]:
class_df = pd.read_sql("SELECT * FROM class", conn)
class_df

,ID_Class,ID_Course,Section,StartDate,EndDate
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


In [7]:
sa_df = pd.read_sql("SELECT * FROM staffassignment", conn)
sa_df

,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate
0,1,1,1,Teacher Assistant,2020-03-16,None
1,2,4,1,Teacher Assistant,2020-03-16,None
2,3,3,1,Instructor,2020-03-16,None


In [8]:
staff_data_df.rename(columns={
    'EmployeeID':'EmployeeID__c',
    'LastName':'LastName__c',
    'FirstName':'FirstName__c',
    'MiddleName':'MiddleName__c', 
    'BirthDate':'BirthDate__c'  
}, inplace=True)
staff_data_df

,ID_Staff,EmployeeID__c,LastName__c,FirstName__c,MiddleName__c,BirthDate__c
0,1,000184220,Wimberly,Sam,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Popelka,Sarah,Nisan,1994-06-07


In [ ]:
staff_data_df['BirthDate__c'] = pd.to_datetime(staff_data_df['BirthDate__c']).dt.date

In [ ]:
staff_data_df = staff_data_df[['EmployeeID__c', 'LastName__c', 'FirstName__c', 'MiddleName__c','BirthDate__c']]
staff_data_df

In [ ]:
staff_data_records = staff_data_df.to_dict('records')
staff_data_records

In [ ]:
for rec in staff_data_records:

    record = {
        'EmployeeID__c': rec['EmployeeID__c'],
        'LastName__c': rec['LastName__c'],
        'FirstName__c': rec['FirstName__c'],
        'MiddleName__c': rec['MiddleName__c'],
        'BirthDate__c': str(rec['BirthDate__c'])
    }
    
    try:
        sf.Staff__c.create(record)
    except Exception as e:
        print(e)
        

## Create Staff Lookup Table
You will use this later to crosswalk the course code with the primary key from the `Staff` table

In [11]:
staff_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
data = sf.query_all_iter("SELECT EmployeeID__c, Name FROM Staff__c")
for row in data:
    rec = {
        'ID_Staff': row['Name'],
        'EmployeeID__c': row['EmployeeID__c']
    }
    staff_lookup_list.append(rec)

In [12]:
staff_lookup_df = pd.DataFrame(staff_lookup_list)
staff_lookup_df

,ID_Staff,EmployeeID__c
0,a0G3h000001K4EJ,000184220
1,a0G3h000001K4EO,000130109
2,a0G3h000001K4ET,000160655
3,a0G3h000001K4EY,000159108


## Create Class Lookup Table


In [20]:
class_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
data = sf.query_all_iter("SELECT Section__c, Name FROM class__c")
for row in data:
    rec = {
        'class ID': row['Name'],
        'Section__c': row['Section__c']
    }
    class_lookup_list.append(rec)

In [21]:
class_lookup_df = pd.DataFrame(class_lookup_list)
class_lookup_df

,class ID,Section__c
0,a083h0000010ByB,GWDC201805DATA3
1,a083h0000010ByL,GWARL201905UIUX3
2,a083h0000010By6,GWU-ARL-DATA-PT-09-0
3,a083h0000010ByG,GWARL201905WEB3


In [15]:
query = '''
    SELECT
        sa.*
        ,c.Section
        ,s.employeeID
    FROM
        staffassignment sa
        INNER JOIN class c
        ON c.ID_class  = sa.ID_class
        INNER Join staff s 
        on s.id_staff = sa.id_staff
'''
staff_assingment_data_df = pd.read_sql(query, conn)
staff_assingment_data_df

,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate,Section,employeeID
0,1,1,1,Teacher Assistant,2020-03-16,None,GWU-ARL-DATA-PT-09-0,000184220
1,2,4,1,Teacher Assistant,2020-03-16,None,GWU-ARL-DATA-PT-09-0,000159108
2,3,3,1,Instructor,2020-03-16,None,GWU-ARL-DATA-PT-09-0,000160655


In [35]:
staff_assingment_data_df.rename(columns={
    'Role':'Role__c',
    'StartDate':'StartDate__c',
    'EndDate':'Enddate__c',
    'Section': 'Section__c',
    'employeeID':'EmployeeID__c',
    'ID_Staff':'Staff_ID'
}, inplace=True)

staff_assingment_data_df

,ID_StaffAssignment,Staff_ID,ID_Class,Role__c,StartDate__c,Enddate__c,Section__c,EmployeeID__c
0,1,1,1,Teacher Assistant,2020-03-16,None,GWU-ARL-DATA-PT-09-0,000184220
1,2,4,1,Teacher Assistant,2020-03-16,None,GWU-ARL-DATA-PT-09-0,000159108
2,3,3,1,Instructor,2020-03-16,None,GWU-ARL-DATA-PT-09-0,000160655


## Join the Class DataFrame with the Course lookup table
This join is necessary to successfully lookup the foreign key for the Course table 

In [36]:
class_lookup_df

,class ID,Section__c
0,a083h0000010ByB,GWDC201805DATA3
1,a083h0000010ByL,GWARL201905UIUX3
2,a083h0000010By6,GWU-ARL-DATA-PT-09-0
3,a083h0000010ByG,GWARL201905WEB3


In [37]:
staff_lookup_df

,ID_Staff,EmployeeID__c
0,a0G3h000001K4EJ,000184220
1,a0G3h000001K4EO,000130109
2,a0G3h000001K4ET,000160655
3,a0G3h000001K4EY,000159108


In [38]:
staff_assingment_data_df

,ID_StaffAssignment,Staff_ID,ID_Class,Role__c,StartDate__c,Enddate__c,Section__c,EmployeeID__c
0,1,1,1,Teacher Assistant,2020-03-16,None,GWU-ARL-DATA-PT-09-0,000184220
1,2,4,1,Teacher Assistant,2020-03-16,None,GWU-ARL-DATA-PT-09-0,000159108
2,3,3,1,Instructor,2020-03-16,None,GWU-ARL-DATA-PT-09-0,000160655


In [39]:
staff_a_df = pd.merge(staff_assingment_data_df, class_lookup_df, how='left')
#class_data_df.drop(columns = ['ID_Class','ID_Course','Course__c','Section__c','StartDate__c','Enddate__c','coursecode__c'], inplace=True)

staff_a_df

,ID_StaffAssignment,Staff_ID,ID_Class,Role__c,StartDate__c,Enddate__c,Section__c,EmployeeID__c,class ID
0,1,1,1,Teacher Assistant,2020-03-16,None,GWU-ARL-DATA-PT-09-0,000184220,a083h0000010By6
1,2,4,1,Teacher Assistant,2020-03-16,None,GWU-ARL-DATA-PT-09-0,000159108,a083h0000010By6
2,3,3,1,Instructor,2020-03-16,None,GWU-ARL-DATA-PT-09-0,000160655,a083h0000010By6


In [42]:
staff_a_final_df = pd.merge(staff_a_df, staff_lookup_df, how='left')

In [43]:
staff_a_final_df.drop(columns = ['ID_StaffAssignment','Staff_ID','ID_Class','Section__c','EmployeeID__c'], inplace=True)

staff_a_final_df

,Role__c,StartDate__c,Enddate__c,class ID,ID_Staff
0,Teacher Assistant,2020-03-16,None,a083h0000010By6,a0G3h000001K4EJ
1,Teacher Assistant,2020-03-16,None,a083h0000010By6,a0G3h000001K4EY
2,Instructor,2020-03-16,None,a083h0000010By6,a0G3h000001K4ET


In [44]:
staff_a_final_df
staff_a_final_df['StartDate__c'] = pd.to_datetime(staff_a_final_df['StartDate__c']).dt.date
staff_a_final_df['Enddate__c'] = pd.to_datetime(staff_a_final_df['Enddate__c']).dt.date

staff_a_final_df.head()


,Role__c,StartDate__c,Enddate__c,class ID,ID_Staff
0,Teacher Assistant,2020-03-16,NaT,a083h0000010By6,a0G3h000001K4EJ
1,Teacher Assistant,2020-03-16,NaT,a083h0000010By6,a0G3h000001K4EY
2,Instructor,2020-03-16,NaT,a083h0000010By6,a0G3h000001K4ET


In [46]:
staff_a_final_df=staff_a_final_df.rename(columns={
    'class ID':'class_ID__c',
    'ID_Staff':'ID_Staff__c'})
staff_a_final_df

,Role__c,StartDate__c,Enddate__c,class_ID__c,ID_Staff__c
0,Teacher Assistant,2020-03-16,NaT,a083h0000010By6,a0G3h000001K4EJ
1,Teacher Assistant,2020-03-16,NaT,a083h0000010By6,a0G3h000001K4EY
2,Instructor,2020-03-16,NaT,a083h0000010By6,a0G3h000001K4ET


In [47]:
staff_assignment_records = staff_a_final_df.to_dict(orient='records')
staff_assignment_records

[{'Role__c': 'Teacher Assistant',
  'StartDate__c': datetime.date(2020, 3, 16),
  'Enddate__c': NaT,
  'class_ID__c': 'a083h0000010By6',
  'ID_Staff__c': 'a0G3h000001K4EJ'},
 {'Role__c': 'Teacher Assistant',
  'StartDate__c': datetime.date(2020, 3, 16),
  'Enddate__c': NaT,
  'class_ID__c': 'a083h0000010By6',
  'ID_Staff__c': 'a0G3h000001K4EY'},
 {'Role__c': 'Instructor',
  'StartDate__c': datetime.date(2020, 3, 16),
  'Enddate__c': NaT,
  'class_ID__c': 'a083h0000010By6',
  'ID_Staff__c': 'a0G3h000001K4ET'}]

## Insert `Staff Assignment` Records into Salesforce

In [49]:
for rec in staff_assignment_records:
 
    record = {
        'Role__c': rec['Role__c'],
        'class_ID__c': rec['class_ID__c'],
        'StartDate__c': str(rec['StartDate__c']),
        'ID_Staff__c':rec['ID_Staff__c']
    }
    
    try:
        sf.staffassignment__c.create(record)
    except Exception as e:
        print(e)